# Работа с GigaChat Vision
В этом ноутбуке мы рассмотрим работу с GigaChat Vision

Наши действия:
1. Загружаем фото на S3 хранилище GigaChat
2. Просим GigaChat проанализировать фотографии, выделить сущности на них и написать описание к фото
3. Выдать ответ в виде JSON, после чего преобразуем этот JSON в Pydantic модель

Тестировать будем на этих фото
![фото 1](cat.jpg)
![фото 2](sea.jpg)

In [7]:
!pip install langchain_gigachat langchain -q


[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: pip install --upgrade pip


In [1]:
import getpass
import os

if "GIGACHAT_CREDENTIALS" not in os.environ:
    os.environ["GIGACHAT_CREDENTIALS"] = getpass.getpass("Credentials от GigaChat")

In [2]:
from langchain_gigachat.chat_models import GigaChat

llm = GigaChat(
    base_url="https://gigachat-preview.devices.sberbank.ru/api/v1",
    temperature=0.1,
    verify_ssl_certs=False,
    timeout=6000,
    model="GigaChat-Pro-preview"
)

In [3]:
from typing import List, Optional

from langchain.output_parsers import PydanticOutputParser
from langchain_core.messages import HumanMessage
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from pydantic import BaseModel, Field, validator
from langchain_core.runnables import RunnableLambda, RunnableParallel


class Photo(BaseModel):
    """Информация о фото"""

    content: str = Field(..., description="Что изображено на фото? 1-3 слова")
    description: str = Field(..., description="Опиши детальнее фото")


# Set up a parser
parser = PydanticOutputParser(pydantic_object=Photo)


def _get_messages_from_url(url: str):
    return {
        "history": [
            HumanMessage(content="", additional_kwargs={"attachments": [url]}),
        ]
    }


# Prompt
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "Определи объект на фото. Ответь на запрос пользователя в формате JSON. Schema Information: \n{format_instructions}",
        ),
        MessagesPlaceholder("history"),
    ]
).partial(format_instructions=parser.get_format_instructions())

chain = RunnableLambda(_get_messages_from_url) | prompt | llm | parser

В итоге мы создали LCEL цепочку, в которую мы можем передавать id загруженных файлов и в ответ получать Pydantic модели

In [4]:
# Пример работы
file = llm.upload_file(open("cat.jpg", "rb"))
file2 = llm.upload_file(open("sea.jpg", "rb"))
chain.batch([file.id_, file2.id_])

[Photo(content='котенок', description='белый котенок с черно-серыми пятнами на спине и хвосте'),
 Photo(content='пляж', description='На фотографии изображен каменистый пляж у подножия холма с домами и маяком. Море волнуется, создавая белые гребни волн.')]